In [187]:
import pandas as pd
import math
import numpy as np

In [153]:
def clean_addr(add):
    add = add.replace('    ', ' ') # 4 space
    add = add.replace('   ', ' ') # 3 space
    add = add.replace('  ', ' ') # 2 space
    add = add.replace(' - ', '-') # spaces around dash
    add = add.upper()
    add = add.replace(' AVE,', ' AVENUE,')
    add = add.replace(' AVE ', ' AVENUE ')
    add = add.replace(' AV,', ' AVENUE,') 
    add = add.replace(' AV ', ' AVENUE ')
    add = add.replace(' ST,', ' STREET,')
    add = add.replace(' ST ', ' STREET ')
    add = add.replace(' PL,', ' PLACE,')
    add = add.replace(' PL ', ' PLACE ')
    add = add.replace(' RD,', ' ROAD,')
    add = add.replace(' RD ', ' ROAD ')
    add = add.replace(' BLVD,', ' BOULEVARD,')
    add = add.replace(' BLVD ', ' BOULEVARD ')
    add = add.replace('FIRST', '1')
    add = add.replace('1ST', '1')
    add = add.replace('SECOND', '2')
    add = add.replace('2ND', '1')
    add = add.replace('THIRD', '3')
    add = add.replace('3RD', '3')
    add = add.replace('FOURTH', '4')
    add = add.replace('4TH', '4')
    add = add.replace('FIFTH', '5')
    add = add.replace('5TH', '5')
    add = add.replace('SIXTH', '6')
    add = add.replace('6TH', '6')
    add = add.replace('SEVENTH', '7')
    add = add.replace('7TH', '7')
    add = add.replace('EIGHTH', '8')
    add = add.replace('8TH', '8')
    add = add.replace('NINTH', '9')
    add = add.replace('9TH', '9')
    add = add.replace(' E ', ' EAST ')
    add = add.replace(' E,', ' EAST,')
    add = add.replace(' W ', ' WEST ')
    add = add.replace(' W,', ' WEST,')
    add = add.replace(' N ', ' NORTH ')
    add = add.replace(' N,', ' NORTH,')
    add = add.replace(' S ', ' SOUTH ')
    add = add.replace(' S,', ' SOUTH,')
    return add

In [154]:
complaints = ['Food%20Poisoning',
              'Food%20Establishment']

In [155]:
# get food-related complaints from NYC 311 API
df_311 = pd.DataFrame()
for complaint in complaints:
    
    complaint = complaint
    limit = 50000
    # page through API results (max 50,000)
    offset = 0
    # any complaint opened after the start of 2012
    start_date = "2012-01-01T00:00:00.000"
    
    while True:
        query_string = "".join(
        """https://data.cityofnewyork.us/resource/fhrw-4uyv.json?
        complaint_type={}&$
        limit={}&$
        offset={}&$
        order=:id&$
        where=created_date>'{}'"""\
        .format(complaint, limit, offset, start_date).splitlines()).replace("    ", "")
        print 'complaint: {}, offset: {}'.format(complaint, offset)
        
        data = pd.read_json(query_string)
        df_311 = df_311.append(data)

        offset += 1
        # if ever a response comes back with less than maximum data, last page has been reached
        if len(data) < limit:
            break

complaint: Food%20Poisoning, offset: 0
complaint: Food%20Establishment, offset: 0


In [156]:
# cleaning 311
# incidents with real addresses only
df_311 = df_311[df_311.address_type=='ADDRESS']
# exclude missing addresses
df_311 = df_311[map(lambda x: type(x)!=float, df_311['incident_address'])]
# restaurant locations only
df_311 = df_311[map(lambda typ: typ in ['Restaurant/Bar/Deli/Bakery', 'Soup Kitchen'], df_311.location_type)]

In [157]:
df_311.drop([u'address_type', u'agency', u'agency_name', 
       u'closed_date', u'community_board', u'created_date',
       u'cross_street_1', u'cross_street_2', u'due_date',
       u'intersection_street_1', u'intersection_street_2', u'landmark',
       u'park_borough', u'park_facility_name',
       u'resolution_action_updated_date', u'resolution_description',
       u'school_address', u'school_city', u'school_code', u'school_name',
       u'school_not_found', u'school_number', u'school_phone_number',
       u'school_region', u'school_state', u'school_zip', u'status',
       u'street_name', u'unique_key', u'x_coordinate_state_plane',
       u'y_coordinate_state_plane'], axis=1, inplace=True)

In [158]:
def combine_addr(row):
    """
    combine address, borough, and zip, into a single string address
    """
    return ', '.join([str(row['incident_address']),
                      str(row['borough']),
                      str(row['incident_zip'])])

In [159]:
df_311.loc[:, 'addr'] = df_311.apply(combine_addr, axis=1)

In [160]:
# STANDARDIZE the 311 addresses in preparation for merging
df_311.loc[:, 'addr'] = map(clean_addr, df_311.loc[:, 'addr'])

In [163]:
df_311 = pd.DataFrame(df_311.groupby('addr').count().loc[:, 'descriptor'])

In [164]:
# get original inspection data, with addresses
insp = pd.read_csv('data/spatial_merged_3_fs_features.csv')

In [165]:
# exclude missing inspectionaddresses
insp = insp[map(lambda x: type(x)!=float, insp['loc'])]

In [166]:
# STANDARDIZE the 311 addresses in preparation for merging
insp.loc[:, 'loc'] = map(clean_addr, insp.loc[:, 'loc'])

In [167]:
df_311[:10]

addr
1 8 AVENUE, BROOKLYN, 11217                   1
1 ABINGDON SQUARE, MANHATTAN, 10014           3
1 ASTOR PLACE, MANHATTAN, 10003               2
1 AVENUE, MANHATTAN, 10003                    2
1 AVENUE, MANHATTAN, 10009                    2
1 BAY CLUB DRIVE, QUEENS, 11360               1
1 BAY STREET LANDING, STATEN ISLAND, 10301    1
1 BAY STREET, STATEN ISLAND, 10301            5
1 BEARD STREET, BROOKLYN, 11231               6
1 BEDFORD PARK BOULEVARD, BRONX, 10468        2
Name: descriptor, dtype: int64

In [168]:
insp['loc'][:10]

0    75 CHRISTOPHER STREET, MANHATTAN, 10014
1             461 37 STREET, BROOKLYN, 11232
2             987 2 AVENUE, MANHATTAN, 10022
3      925 MANHATTAN AVENUE, BROOKLYN, 11222
4    55 CHRISTOPHER STREET, MANHATTAN, 10014
5         17 PRINCE STREET, MANHATTAN, 10012
6     118 RIVINGTON STREET, MANHATTAN, 10002
7        23 WEST 45 STREET, MANHATTAN, 10036
8             977 2 AVENUE, MANHATTAN, 10022
9       765 CONCOURSE VLG WEST, BRONX, 10451
Name: loc, dtype: object

In [169]:
len(insp)

25926

In [170]:
len(df_311)

20590

In [190]:
# left join on standardized addresses
merged = insp.merge(df_311, left_on='loc', right_index=True, how='left')

In [191]:
merged = merged.loc[:, ['CAMIS', 'descriptor']]
merged.columns = ['CAMIS', '311_complaint_count']
print '{} matches'.format(sum(map(lambda x: np.isfinite(x), merged['311_complaint_count'])))

10176 matches


In [196]:
merged.head()

,CAMIS,311_complaint_count
0,41014823,2.0
1,40364889,1.0
2,40571896,NaN
3,50004740,NaN
4,41331076,NaN


In [197]:
# write to file
merged.to_csv('data/311_complaint.csv', index=False)